# L5: Content Creation at Scale

<p style="background-color:#fff6e4; padding:15px; border-width:3px; border-color:#f5ecda; border-style:solid; border-radius:6px"> ⏳ <b>Note <code>(Kernel Starting)</code>:</b> This notebook takes about 30 seconds to be ready to use. You may start and watch the video while you wait.</p>

## Initial Imports

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

# Load environment variables
from helper import load_env
load_env()

import os
import yaml
from crewai import Agent, Task, Crew

<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; <b>Access <code>requirements.txt</code> and <code>helper.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>. For more help, please see the <em>"Appendix - Tips and Help"</em> Lesson.</p>

## Creating Structured Output

In [2]:
from pydantic import BaseModel, Field
from typing import List

class SocialMediaPost(BaseModel):
    platform: str = Field(..., description="The social media platform where the post will be published (e.g., Twitter, LinkedIn).")
    content: str = Field(..., description="The content of the social media post, including any hashtags or mentions.")

class ContentOutput(BaseModel):
    article: str = Field(..., description="The article, formatted in markdown.")
    social_media_posts: List[SocialMediaPost] = Field(..., description="A list of social media posts related to the article.")

## Loading Tasks and Agents YAML files

In [3]:
# Define file paths for YAML configurations
files = {
    'agents': 'config/agents.yaml',
    'tasks': 'config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

## Importing CrewAI Tools

In [4]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool

## Setup Multi LLM models

> Note: The 'llama-3.1-70b-versatil' model was deprecated. As a result, we updated it to the 'llama-3.3-70b-versatil' model in January 2025.

In [5]:
os.environ['OPENAI_MODEL_NAME'] = 'gpt-4o-mini'
groq_llm = "groq/llama-3.3-70b-versatile"

## Creating Crew, Agents, and Tasks

In [6]:
# Creating Agents
market_news_monitor_agent = Agent(
    config=agents_config['market_news_monitor_agent'],
    tools=[SerperDevTool(), ScrapeWebsiteTool()],
    llm=groq_llm,
)

data_analyst_agent = Agent(
    config=agents_config['data_analyst_agent'],
    tools=[SerperDevTool(), WebsiteSearchTool()],
    llm=groq_llm,
)

content_creator_agent = Agent(
    config=agents_config['content_creator_agent'],
    tools=[SerperDevTool(), WebsiteSearchTool()],
)

quality_assurance_agent = Agent(
    config=agents_config['quality_assurance_agent'],
)

# Creating Tasks
monitor_financial_news_task = Task(
    config=tasks_config['monitor_financial_news'],
    agent=market_news_monitor_agent
)

analyze_market_data_task = Task(
    config=tasks_config['analyze_market_data'],
    agent=data_analyst_agent
)

create_content_task = Task(
    config=tasks_config['create_content'],
    agent=content_creator_agent,
    context=[monitor_financial_news_task, analyze_market_data_task]
)

quality_assurance_task = Task(
    config=tasks_config['quality_assurance'],
    agent=quality_assurance_agent,
    output_pydantic=ContentOutput
)

# Creating Crew
content_creation_crew = Crew(
    agents=[
        market_news_monitor_agent,
        data_analyst_agent,
        content_creator_agent,
        quality_assurance_agent
    ],
    tasks=[
        monitor_financial_news_task,
        analyze_market_data_task,
        create_content_task,
        quality_assurance_task
    ],
    verbose=True
)

2025-08-16 12:17:25,993 - 140650214194048 - sqlite.py-sqlite:111 - WARNING: ⚠️ It looks like you upgraded from a version below 0.6 and could benefit from vacuuming your database. Run chromadb utils vacuum --help for more information.


## Kicking off the Crew

In [7]:
result = content_creation_crew.kickoff(inputs={
  'subject': 'Inflation in the US and the impact on the stock market in 2024'
})

# Agent: Lead Market Analyst
## Task: Monitor and analyze the latest news and updates related to the financial markets, with a particular focus on Inflation in the US and the impact on the stock market in 2024. Identify and summarize the most relevant and impactful news items that could influence market trends or investor decisions. Utilize financial news APIs and real-time market data tools to gather up-to-date information. Focus on detecting trends, regulatory changes, or significant economic indicators that directly relate to Inflation in the US and the impact on the stock market in 2024.



# Agent: Lead Market Analyst
## Thought: To create a detailed summary report on the most impactful financial news and updates related to Inflation in the US and the impact on the stock market in 2024, I should start by gathering the latest information available on the internet regarding inflation trends, regulatory changes, and significant economic indicators in the US that could affect the stoc



# Agent: Lead Market Analyst
## Final Answer: 
Consumer prices rose 2.7% annually in July, less than expected, amid tariff worries. The seasonally adjusted 0.2% for the month and 2.7% on a 12-month basis, the Bureau of Labor Statistics reported Tuesday. Excluding food and energy, the core CPI increased 0.3% for the month and 3.1% from a year ago. The report comes at both a critical time for the economy and the BLS itself, which has come under Trump's criticism for what he has charged is political bias against him.

Following the report, traders ramped up bets that the Federal Reserve would start reducing rates again in September. Futures market pricing is pointing strongly to a Fed rate cut in September. Traders increased the implied odds for a September move following the release, and also put the chances of another reduction in October at about 67%, up from 55% the day before, according to the CME Group's FedWatch tool.

Tariffs did appear to show up in several categories, but othe

Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  4.73it/s]




# Agent: Chief Data Strategist
## Thought: Let's fix this and continue.
## Using tool: Search in a specific website
## Tool Input: 
"{\"search_query\": \"inflation impact on stock market 2024\", \"website\": \"https://www.cnbc.com/\"}"
## Tool Output: 
Relevant Content:
outlook. In this scenario, continued geopolitical conflicts and trade actions cause inflation to stay higher for longer, driving the Federal Reserve to hike rates.1 Scenarios Baseline (70%): Real GDP growth slowed overall in the first quarter of 2024, coming in very close to our previous forecast’s expectations.2 We expect GDP growth will continue to moderate through the second half of this year and the start of the next, but the story is still positive overall. Consumer spending is forecasted to rise 2.3% this year, up from the 2.2% increase in 2023. Business investment is expected to increase 3% in 2024, down from 4.5% last year. Government spending is forecasted to rise 2.5%. Given that the US economy is expected t



# Agent: Creative Content Director
## Thought: I need to gather current insights and trends on inflation in the US and its impact on the stock market for 2024. This information will help in creating high-quality content that resonates with the target audience.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"US inflation impact on stock market 2024\"}"
## Tool Output: 

Search results: Title: Slower Growth, Higher Inflation And S&P 500 All-time Highs
Link: https://www.jpmorgan.com/insights/markets/top-market-takeaways/tmt-slower-growth-higher-inflation-and-s-and-p-five-hundred-all-time-highs
Snippet: Amid an uneven macro backdrop, we explore where investors are finding opportunity.
---
Title: U.S. Inflation Report Shows Effects of Trump's Tariffs
Link: https://www.nytimes.com/live/2025/08/12/business/cpi-inflation-tariffs-fed
Snippet: In June, core inflation rose 0.2 percent from the previous month, or 2.9 percent from July 2024.
---
Title: Which Equity Sector



# Agent: Chief Content Officer
## Final Answer: 
```markdown
# Navigating the Turbulent Waters of Inflation in 2024

## Introduction
As we step into 2024, the economic landscape remains a whirlwind of changes, especially concerning inflation rates in the US. Recent reports indicate a consumer price increase of **2.7% year-over-year**, which presents a mixed bag of challenges and opportunities for investors. Understanding these inflation trends and their implications on the stock market is crucial for crafting effective investment strategies in this ever-evolving environment. Let’s delve into these insights and explore how investors can navigate the potentially turbulent waters ahead.

## Current State of Inflation
In July 2024, inflation appeared lower than expected at **2.7%**, sparking speculation regarding potential **rate cuts by the Federal Reserve**. **Ellen Zentner**, chief economic strategist for Morgan Stanley, noted that while inflation is on the rise, it hasn't surged beyo

## Social Content

In [8]:
import textwrap

posts = result.pydantic.dict()['social_media_posts']
for post in posts:
    platform = post['platform']
    content = post['content']
    print(platform)
    wrapped_content = textwrap.fill(content, width=50)
    print(wrapped_content)
    print('-' * 50)

Twitter
📈 U.S. inflation sits at **2.7%**! Strong market
reaction post-report, with the **#Dow** up **200
pts**! Investors, prepare for volatility and stay
strategic. #InvestSmart #StockMarket2024
--------------------------------------------------
LinkedIn
As inflation stabilizes at **2.7%**, the stock
market responds positively. Key opportunities lie
ahead, but with increased volatility expected. Our
analysis highlights which sectors may be your best
bets. Check out our full blog for insights! [Read
More](#)
--------------------------------------------------
Instagram
🆕 Inflation is shaping our investments! 📊 With a
**2.7% CPI impact**, how are you adjusting your
portfolio? Dive into our latest insights to
navigate the market effectively! #Inflation
#FinanceTips
--------------------------------------------------


## Blog Post

In [9]:
from IPython.display import display, Markdown
display(Markdown(result.pydantic.dict()['article']))

# Navigating the Turbulent Waters of Inflation in 2024

## Introduction
As we step into 2024, the economic landscape remains a whirlwind of changes, especially concerning inflation rates in the US. Recent reports indicate a consumer price increase of **2.7% year-over-year**, which presents a mixed bag of challenges and opportunities for investors. Understanding these inflation trends and their implications on the stock market is crucial for crafting effective investment strategies in this ever-evolving environment. Let’s delve into these insights and explore how investors can navigate the potentially turbulent waters ahead.

## Current State of Inflation
In July 2024, inflation appeared lower than expected at **2.7%**, sparking speculation regarding potential **rate cuts by the Federal Reserve**. **Ellen Zentner**, chief economic strategist for Morgan Stanley, noted that while inflation is on the rise, it hasn't surged beyond expectations. This perspective is leading to a cautious optimism in market performance as investors take stock of their options.

### Core CPI Insights
- The **core Consumer Price Index (CPI)**, which excludes food and energy, recorded a **0.3% monthly increase** and a **3.1% annual rise**. These metrics indicate persistent inflationary pressures but remain within manageable limits, providing a favorable backdrop for equities in certain sectors.

## Stock Market Reactions
In response to the recent inflation report, the stock market reacted positively, marking a notable rebound:
- **Dow Jones**: Up **200 points** (+0.7%)
- **S&P 500**: Up **0.8%**
- **Nasdaq**: Up **1%**

Traders swiftly adjusted their outlook, boosting expectations for **rate cuts** by the Federal Reserve, thus fostering a bullish sentiment amongst investors. This demonstrates the market's sensitivity to macroeconomic indicators and the importance of staying informed.

## Key Insights for Investors
1. **Stay Agile**: Given the anticipated volatility, maintaining a flexible investment strategy is crucial. This might include diversifying portfolios and being prepared to pivot in response to market changes.
2. **Watch Sectors**: Equities historically outperform inflation in low-growth conditions. Focus on sectors less sensitive to commodity price changes, such as **technology and healthcare**, which are poised for growth even in uncertain economic times.
3. **Geopolitical Awareness**: Issues such as tariffs and international relations could significantly impact long-term inflation trends. Investors are encouraged to account for geopolitical risk in their strategies, as it may introduce further unpredictability within the market.

## Data Visualizations
- **Figure 1**: Projected US GDP Growth Rate (2024-2028) — A line graph illustrating sustained growth before a projected slowdown in 2025.
- **Figure 2**: CPI Inflation Trends (2024-2025) — A bar chart showcasing expected CPI trends against historical data.
- **Figure 3**: Federal Reserve Rate Decisions — A timeline of anticipated interest rate changes in response to evolving inflation metrics.

## Social Media Updates

### Twitter
📈 U.S. inflation sits at **2.7%**! Strong market reaction post-report, with the **#Dow** up **200 pts**! Investors, prepare for volatility and stay strategic. #InvestSmart #StockMarket2024

### LinkedIn
As inflation stabilizes at **2.7%**, the stock market responds positively. Key opportunities lie ahead, but with increased volatility expected. Our analysis highlights which sectors may be your best bets. Check out our full blog for insights! [Read More](#)

### Instagram
🆕 Inflation is shaping our investments! 📊 With a **2.7% CPI impact**, how are you adjusting your portfolio? Dive into our latest insights to navigate the market effectively! #Inflation #FinanceTips

## Conclusion
Understanding the nuances of inflation in the US is essential for strategic investment planning in 2024. The current volatility of the stock market underscores the importance of informed decision-making. By remaining vigilant of financial trends and adapting investment strategies in alignment with economic signals, investors can better prepare for the challenges and opportunities that lie ahead.

## Content Recommendations
1. **Articles Series**: Exploring inflation's impact on specific industries — technology, consumer goods, and healthcare.
2. **Video Content**: A visual narrative explaining inflation's influence on interest rates and investment strategies.
3. **Podcast Episodes**: Discussions with financial experts on the evolving landscape of investing amid inflation concerns.

By providing this high-value content, we can empower investors to make informed decisions and foster a deeper understanding of the financial landscape.